# pip Installtions

In [1]:
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
pip install transformers datasets evaluate

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


# Imports

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, matthews_corrcoef
from transformers.keras_callbacks import PushToHubCallback
from huggingface_hub import notebook_login

from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
from transformers import create_optimizer
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification
from transformers.keras_callbacks import KerasMetricCallback
from tensorflow.keras.callbacks import ModelCheckpoint
from keras import backend as K

2024-01-15 11:00:53.401442: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-15 11:00:53.437282: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-15 11:00:53.620760: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-15 11:00:53.622599: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-15 11:00:54.453378: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

# GPU check

In [5]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


2024-01-15 11:00:58.529359: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-15 11:00:58.530073: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Jan 15 11:01:18 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   49C    P8    13W / 115W |      6MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Hugging Face Login

In [8]:
notebook_login()

# SET TOKENIZER

In [7]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")

In [8]:
dataframe_completo = pd.read_csv("Datasets/out.csv")
ds_completo = Dataset.from_pandas(dataframe_completo)
ds_completo

Dataset({
    features: ['source', 'question', 'answer', 'context', 'language', 'label'],
    num_rows: 15164
})

In [11]:
y = dataframe_completo.loc[:,"label"]
X = dataframe_completo.loc[:,"question"]

In [12]:
X_train, X_to_split, y_train, y_to_split = train_test_split(X, y, test_size=0.2, random_state=7, stratify=y)

In [13]:
X_validation, X_test, y_validation, y_test = train_test_split(X_to_split, y_to_split, test_size=0.5, random_state=7, stratify=y_to_split)

In [14]:
train_df = pd.concat([X_train, y_train], axis=1)
validation_df = pd.concat([X_validation, y_validation], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

In [15]:
train_df

,question,label
2404,Qual o dano nas algas está sendo causado pelo ...,1.0
12654,Em que setor os empregos começam a diminuir?,0.0
13630,Quando foi estabelecida a Direção da Bibliotec...,0.0
2560,Em quais as áreas é esperado que o ruído antro...,1.0
4582,Qual foi o tamanho da população da ilha de São...,1.0
...,...,...
7108,O Parque Nacional Monte Pascoal Ã© em qual bioma,0.0
10031,Que tipo de floresta Ã© a selva amazÃ´nica?,0.0
12231,Quantos colonos originais se estabeleceram em ...,0.0
379,Onde as pinças de Mithraculis foram coletadas?,1.0


In [16]:
train_ds = Dataset.from_pandas(train_df)
validation_ds = Dataset.from_pandas(validation_df)
test_ds = Dataset.from_pandas(test_df)

In [17]:
train_ds

Dataset({
    features: ['question', 'label', '__index_level_0__'],
    num_rows: 12131
})

In [18]:
def preprocess_function(examples):
    return tokenizer(examples["question"], truncation=True)

In [19]:
tokenized_train_ds = train_ds.map(preprocess_function, batched=True)
tekenized_validation_ds = validation_ds.map(preprocess_function, batched=True)
tokenized_test_ds = test_ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/12131 [00:00<?, ? examples/s]

Map:   0%|          | 0/1516 [00:00<?, ? examples/s]

Map:   0%|          | 0/1517 [00:00<?, ? examples/s]

In [20]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [21]:
# f1_metric = evaluate.load("f1")
# accuracy = evaluate.load("accuracy")

In [22]:
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
#     return accuracy.compute(predictions=predictions, references=labels)

In [23]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [24]:
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

# MODEL 1

In [57]:
batch_size = 16
num_epochs = 5
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [58]:
model = TFAutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)
tf_train_set = model.prepare_tf_dataset(
    tokenized_train_ds,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tekenized_validation_ds,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
model.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m])  # No loss argument!
# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)
push_to_hub_callback = PushToHubCallback(
    output_dir="IC_primeiro",
    tokenizer=tokenizer,
)
checkpoint = ModelCheckpoint(filepath = 'model1.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                             monitor = 'val_accuracy',
                             verbose = 1,
                             save_best_only = True,
                             mode = 'max')
callbacks = [checkpoint, push_to_hub_callback]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\gusta\Documents\IC\IC_primeiro is already a clone of https://huggingface.co/gustavokpc/IC_primeiro. Make sure you pull the latest changes with `repo.git_pull()`.


In [59]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=num_epochs, callbacks=callbacks)

Epoch 1/5
758/758 [==============================] - ETA: 0s - loss: 0.3533 - accuracy: 0.8498 - f1_m: 0.4723 - precision_m: 0.4085 - recall_m: 0.6530
Epoch 1: val_accuracy improved from -inf to 0.90369, saving model to model1.epoch01-accuracy0.90369.tf
INFO:tensorflow:Assets written to: model1.epoch01-accuracy0.90369.tf\assets


INFO:tensorflow:Assets written to: model1.epoch01-accuracy0.90369.tf\assets


758/758 [==============================] - 1522s 2s/step - loss: 0.3533 - accuracy: 0.8498 - f1_m: 0.4723 - precision_m: 0.4085 - recall_m: 0.6530 - val_loss: 0.2424 - val_accuracy: 0.9037 - val_f1_m: 0.5060 - val_precision_m: 0.3909 - val_recall_m: 0.7591
Epoch 2/5
758/758 [==============================] - ETA: 0s - loss: 0.1974 - accuracy: 0.9259 - f1_m: 0.5184 - precision_m: 0.3930 - recall_m: 0.8161
Epoch 2: val_accuracy improved from 0.90369 to 0.92018, saving model to model1.epoch02-accuracy0.92018.tf
INFO:tensorflow:Assets written to: model1.epoch02-accuracy0.92018.tf\assets


INFO:tensorflow:Assets written to: model1.epoch02-accuracy0.92018.tf\assets


758/758 [==============================] - 1564s 2s/step - loss: 0.1974 - accuracy: 0.9259 - f1_m: 0.5184 - precision_m: 0.3930 - recall_m: 0.8161 - val_loss: 0.1978 - val_accuracy: 0.9202 - val_f1_m: 0.5425 - val_precision_m: 0.4014 - val_recall_m: 0.8778
Epoch 3/5
758/758 [==============================] - ETA: 0s - loss: 0.1242 - accuracy: 0.9551 - f1_m: 0.5382 - precision_m: 0.3974 - recall_m: 0.8918
Epoch 3: val_accuracy improved from 0.92018 to 0.92480, saving model to model1.epoch03-accuracy0.92480.tf
INFO:tensorflow:Assets written to: model1.epoch03-accuracy0.92480.tf\assets


INFO:tensorflow:Assets written to: model1.epoch03-accuracy0.92480.tf\assets


758/758 [==============================] - 1556s 2s/step - loss: 0.1242 - accuracy: 0.9551 - f1_m: 0.5382 - precision_m: 0.3974 - recall_m: 0.8918 - val_loss: 0.1970 - val_accuracy: 0.9248 - val_f1_m: 0.5583 - val_precision_m: 0.4106 - val_recall_m: 0.9195
Epoch 4/5
758/758 [==============================] - ETA: 0s - loss: 0.0823 - accuracy: 0.9705 - f1_m: 0.5511 - precision_m: 0.4024 - recall_m: 0.9370
Epoch 4: val_accuracy did not improve from 0.92480
758/758 [==============================] - 1548s 2s/step - loss: 0.0823 - accuracy: 0.9705 - f1_m: 0.5511 - precision_m: 0.4024 - recall_m: 0.9370 - val_loss: 0.2550 - val_accuracy: 0.9116 - val_f1_m: 0.5567 - val_precision_m: 0.4057 - val_recall_m: 0.9330
Epoch 5/5
758/758 [==============================] - ETA: 0s - loss: 0.0532 - accuracy: 0.9812 - f1_m: 0.5544 - precision_m: 0.4027 - recall_m: 0.9558
Epoch 5: val_accuracy did not improve from 0.92480
758/758 [==============================] - 1631s 2s/step - loss: 0.0532 - accuracy

In [ ]:
val_acc = model.history.history['val_accuracy']
val_f1_m = model.history.history['val_f1_m']
print(val_acc)
print(val_f1_m)

[0.8951187133789062, 0.9281002879142761, 0.9320580363273621, 0.9287598729133606, 0.9307388067245483]
[0.5461256504058838, 0.5628437995910645, 0.5655707716941833, 0.5579412579536438, 0.5580920577049255]


# MODEL 2

In [60]:
batch_size = 16
num_epochs = 5
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=5e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [61]:
model2 = TFAutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

tf_train_set2 = model2.prepare_tf_dataset(
    tokenized_train_ds,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set2 = model2.prepare_tf_dataset(
    tekenized_validation_ds,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
model2.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m])  # No loss argument!
# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set2)
push_to_hub_callback = PushToHubCallback(
    output_dir="IC_segundo",
    tokenizer=tokenizer,
)
checkpoint = ModelCheckpoint(filepath = 'model2.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                             monitor = 'val_accuracy',
                             verbose = 1,
                             save_best_only = True,
                             mode = 'max')
callbacks = [checkpoint, push_to_hub_callback]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\gusta\Documents\IC\IC_segundo is already a clone of https://huggingface.co/gustavokpc/IC_segundo. Make sure you pull the latest changes with `repo.git_pull()`.


In [62]:
model2.fit(x=tf_train_set2, validation_data=tf_validation_set2, epochs=num_epochs, callbacks=callbacks)

Epoch 1/5
758/758 [==============================] - ETA: 0s - loss: 0.3576 - accuracy: 0.8399 - f1_m: 0.4604 - precision_m: 0.3607 - recall_m: 0.7042
Epoch 1: val_accuracy improved from -inf to 0.89974, saving model to model2.epoch01-accuracy0.89974.tf
INFO:tensorflow:Assets written to: model2.epoch01-accuracy0.89974.tf\assets


INFO:tensorflow:Assets written to: model2.epoch01-accuracy0.89974.tf\assets


758/758 [==============================] - 1853s 2s/step - loss: 0.3576 - accuracy: 0.8399 - f1_m: 0.4604 - precision_m: 0.3607 - recall_m: 0.7042 - val_loss: 0.2825 - val_accuracy: 0.8997 - val_f1_m: 0.5635 - val_precision_m: 0.4127 - val_recall_m: 0.9300
Epoch 2/5
758/758 [==============================] - ETA: 0s - loss: 0.2012 - accuracy: 0.9274 - f1_m: 0.5204 - precision_m: 0.3849 - recall_m: 0.8616
Epoch 2: val_accuracy improved from 0.89974 to 0.91755, saving model to model2.epoch02-accuracy0.91755.tf
INFO:tensorflow:Assets written to: model2.epoch02-accuracy0.91755.tf\assets


INFO:tensorflow:Assets written to: model2.epoch02-accuracy0.91755.tf\assets


758/758 [==============================] - 1789s 2s/step - loss: 0.2012 - accuracy: 0.9274 - f1_m: 0.5204 - precision_m: 0.3849 - recall_m: 0.8616 - val_loss: 0.2103 - val_accuracy: 0.9175 - val_f1_m: 0.5451 - val_precision_m: 0.3970 - val_recall_m: 0.9095
Epoch 3/5
758/758 [==============================] - ETA: 0s - loss: 0.1312 - accuracy: 0.9511 - f1_m: 0.5451 - precision_m: 0.3969 - recall_m: 0.9273
Epoch 3: val_accuracy improved from 0.91755 to 0.93074, saving model to model2.epoch03-accuracy0.93074.tf
INFO:tensorflow:Assets written to: model2.epoch03-accuracy0.93074.tf\assets


INFO:tensorflow:Assets written to: model2.epoch03-accuracy0.93074.tf\assets


758/758 [==============================] - 1985s 3s/step - loss: 0.1312 - accuracy: 0.9511 - f1_m: 0.5451 - precision_m: 0.3969 - recall_m: 0.9273 - val_loss: 0.2125 - val_accuracy: 0.9307 - val_f1_m: 0.5571 - val_precision_m: 0.4017 - val_recall_m: 0.9523
Epoch 4/5
758/758 [==============================] - ETA: 0s - loss: 0.0871 - accuracy: 0.9690 - f1_m: 0.5547 - precision_m: 0.4007 - recall_m: 0.9557
Epoch 4: val_accuracy did not improve from 0.93074
758/758 [==============================] - 1807s 2s/step - loss: 0.0871 - accuracy: 0.9690 - f1_m: 0.5547 - precision_m: 0.4007 - recall_m: 0.9557 - val_loss: 0.2417 - val_accuracy: 0.9301 - val_f1_m: 0.5565 - val_precision_m: 0.4013 - val_recall_m: 0.9547
Epoch 5/5
758/758 [==============================] - ETA: 0s - loss: 0.0559 - accuracy: 0.9805 - f1_m: 0.5583 - precision_m: 0.4028 - recall_m: 0.9686
Epoch 5: val_accuracy improved from 0.93074 to 0.93272, saving model to model2.epoch05-accuracy0.93272.tf
INFO:tensorflow:Assets writ

INFO:tensorflow:Assets written to: model2.epoch05-accuracy0.93272.tf\assets


758/758 [==============================] - 1830s 2s/step - loss: 0.0559 - accuracy: 0.9805 - f1_m: 0.5583 - precision_m: 0.4028 - recall_m: 0.9686 - val_loss: 0.2533 - val_accuracy: 0.9327 - val_f1_m: 0.5605 - val_precision_m: 0.4028 - val_recall_m: 0.9674


In [79]:
val_acc = model2.history.history['val_accuracy']
val_f1_m = model2.history.history['val_f1_m']
print(val_acc)
print(val_f1_m)

[0.9069920778274536, 0.9241424798965454, 0.9327176809310913]
[0.5674652457237244, 0.5400716662406921, 0.5534441471099854]


# MODEL 3

In [63]:
batch_size = 16
num_epochs = 3
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=3e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [64]:
model3 = TFAutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

tf_train_set3 = model3.prepare_tf_dataset(
    tokenized_train_ds,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set3 = model3.prepare_tf_dataset(
    tekenized_validation_ds,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
model3.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m])  # No loss argument!
# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set3)
push_to_hub_callback = PushToHubCallback(
    output_dir="IC_terceiro",
    tokenizer=tokenizer,
)
checkpoint = ModelCheckpoint(filepath = 'model3.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                             monitor = 'val_accuracy',
                             verbose = 1,
                             save_best_only = True,
                             mode = 'max')
callbacks = [checkpoint, push_to_hub_callback]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\gusta\Documents\IC\IC_terceiro is already a clone of https://huggingface.co/gustavokpc/IC_terceiro. Make sure you pull the latest changes with `repo.git_pull()`.


In [65]:
model3.fit(x=tf_train_set3, validation_data=tf_validation_set3, epochs=num_epochs, callbacks=callbacks)

Epoch 1/3
758/758 [==============================] - ETA: 0s - loss: 0.3546 - accuracy: 0.8475 - f1_m: 0.5020 - precision_m: 0.4252 - recall_m: 0.7323
Epoch 1: val_accuracy improved from -inf to 0.87467, saving model to model3.epoch01-accuracy0.87467.tf
INFO:tensorflow:Assets written to: model3.epoch01-accuracy0.87467.tf\assets


INFO:tensorflow:Assets written to: model3.epoch01-accuracy0.87467.tf\assets


758/758 [==============================] - 1896s 2s/step - loss: 0.3546 - accuracy: 0.8475 - f1_m: 0.5020 - precision_m: 0.4252 - recall_m: 0.7323 - val_loss: 0.3033 - val_accuracy: 0.8747 - val_f1_m: 0.5815 - val_precision_m: 0.4266 - val_recall_m: 0.9558
Epoch 2/3
758/758 [==============================] - ETA: 0s - loss: 0.1757 - accuracy: 0.9344 - f1_m: 0.5389 - precision_m: 0.3987 - recall_m: 0.8912
Epoch 2: val_accuracy improved from 0.87467 to 0.92612, saving model to model3.epoch02-accuracy0.92612.tf
INFO:tensorflow:Assets written to: model3.epoch02-accuracy0.92612.tf\assets


INFO:tensorflow:Assets written to: model3.epoch02-accuracy0.92612.tf\assets


758/758 [==============================] - 2022s 3s/step - loss: 0.1757 - accuracy: 0.9344 - f1_m: 0.5389 - precision_m: 0.3987 - recall_m: 0.8912 - val_loss: 0.2040 - val_accuracy: 0.9261 - val_f1_m: 0.5665 - val_precision_m: 0.4137 - val_recall_m: 0.9427
Epoch 3/3
758/758 [==============================] - ETA: 0s - loss: 0.1004 - accuracy: 0.9641 - f1_m: 0.5542 - precision_m: 0.4036 - recall_m: 0.9415
Epoch 3: val_accuracy did not improve from 0.92612
758/758 [==============================] - 1859s 2s/step - loss: 0.1004 - accuracy: 0.9641 - f1_m: 0.5542 - precision_m: 0.4036 - recall_m: 0.9415 - val_loss: 0.2133 - val_accuracy: 0.9222 - val_f1_m: 0.5610 - val_precision_m: 0.4087 - val_recall_m: 0.9424


In [66]:
val_acc = model3.history.history['val_accuracy']
val_f1_m = model3.history.history['val_f1_m']
print(val_acc)
print(val_f1_m)

[0.8746702075004578, 0.9261213541030884, 0.9221636056900024]
[0.5815335512161255, 0.5665431618690491, 0.5609732270240784]


# MODEL 4

In [71]:
batch_size = 16
num_epochs = 3
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [72]:
model4 = TFAutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

tf_train_set4 = model4.prepare_tf_dataset(
    tokenized_train_ds,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set4 = model4.prepare_tf_dataset(
    tekenized_validation_ds,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
model4.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m]) # No loss argument!
# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set4)
push_to_hub_callback = PushToHubCallback(
    output_dir="IC_quarto",
    tokenizer=tokenizer,
)
checkpoint = ModelCheckpoint(filepath = 'model4.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                             monitor = 'val_accuracy',
                             verbose = 1,
                             save_best_only = True,
                             mode = 'max')
callbacks = [checkpoint, push_to_hub_callback]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\gusta\Documents\IC\IC_quarto is already a clone of https://huggingface.co/gustavokpc/IC_quarto. Make sure you pull the latest changes with `repo.git_pull()`.


In [73]:
model4.fit(x=tf_train_set4, validation_data=tf_validation_set4, epochs=num_epochs, callbacks=callbacks)

Epoch 1/3
758/758 [==============================] - ETA: 0s - loss: 0.3469 - accuracy: 0.8494 - f1_m: 0.4233 - precision_m: 0.3470 - recall_m: 0.6216
Epoch 1: val_accuracy improved from -inf to 0.89446, saving model to model4.epoch01-accuracy0.89446.tf
INFO:tensorflow:Assets written to: model4.epoch01-accuracy0.89446.tf\assets


INFO:tensorflow:Assets written to: model4.epoch01-accuracy0.89446.tf\assets


758/758 [==============================] - 2824s 4s/step - loss: 0.3469 - accuracy: 0.8494 - f1_m: 0.4233 - precision_m: 0.3470 - recall_m: 0.6216 - val_loss: 0.2535 - val_accuracy: 0.8945 - val_f1_m: 0.5613 - val_precision_m: 0.4145 - val_recall_m: 0.9125
Epoch 2/3
758/758 [==============================] - ETA: 0s - loss: 0.1742 - accuracy: 0.9335 - f1_m: 0.5237 - precision_m: 0.3895 - recall_m: 0.8572
Epoch 2: val_accuracy improved from 0.89446 to 0.90172, saving model to model4.epoch02-accuracy0.90172.tf
INFO:tensorflow:Assets written to: model4.epoch02-accuracy0.90172.tf\assets


INFO:tensorflow:Assets written to: model4.epoch02-accuracy0.90172.tf\assets


758/758 [==============================] - 3049s 4s/step - loss: 0.1742 - accuracy: 0.9335 - f1_m: 0.5237 - precision_m: 0.3895 - recall_m: 0.8572 - val_loss: 0.2315 - val_accuracy: 0.9017 - val_f1_m: 0.5765 - val_precision_m: 0.4256 - val_recall_m: 0.9353
Epoch 3/3
758/758 [==============================] - ETA: 0s - loss: 0.1061 - accuracy: 0.9634 - f1_m: 0.5432 - precision_m: 0.3978 - recall_m: 0.9159
Epoch 3: val_accuracy improved from 0.90172 to 0.92348, saving model to model4.epoch03-accuracy0.92348.tf
INFO:tensorflow:Assets written to: model4.epoch03-accuracy0.92348.tf\assets


INFO:tensorflow:Assets written to: model4.epoch03-accuracy0.92348.tf\assets


758/758 [==============================] - 3110s 4s/step - loss: 0.1061 - accuracy: 0.9634 - f1_m: 0.5432 - precision_m: 0.3978 - recall_m: 0.9159 - val_loss: 0.2101 - val_accuracy: 0.9235 - val_f1_m: 0.5596 - val_precision_m: 0.4070 - val_recall_m: 0.9389


In [74]:
val_acc = model4.history.history['val_accuracy']
val_f1_m = model4.history.history['val_f1_m']
print(val_acc)
print(val_f1_m)

[0.8944591283798218, 0.9017150402069092, 0.9234828352928162]
[0.5612534880638123, 0.5765007138252258, 0.5595895648002625]


# MODEL 5

In [67]:
batch_size = 16
num_epochs = 3
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=1e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [68]:
model5 = TFAutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

tf_train_set5 = model5.prepare_tf_dataset(
    tokenized_train_ds,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set5 = model5.prepare_tf_dataset(
    tekenized_validation_ds,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
model5.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m])  # No loss argument!
# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set5)
push_to_hub_callback = PushToHubCallback(
    output_dir="IC_quinto",
    tokenizer=tokenizer,
)
checkpoint = ModelCheckpoint(filepath = 'model5.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                             monitor = 'val_accuracy',
                             verbose = 1,
                             save_best_only = True,
                             mode = 'max')
callbacks = [checkpoint, push_to_hub_callback]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\gusta\Documents\IC\IC_quinto is already a clone of https://huggingface.co/gustavokpc/IC_quinto. Make sure you pull the latest changes with `repo.git_pull()`.


In [69]:
model5.fit(x=tf_train_set5, validation_data=tf_validation_set5, epochs=num_epochs, callbacks=callbacks)

Epoch 1/3
758/758 [==============================] - ETA: 0s - loss: 0.4076 - accuracy: 0.8160 - f1_m: 0.5002 - precision_m: 0.3900 - recall_m: 0.7694
Epoch 1: val_accuracy improved from -inf to 0.88588, saving model to model5.epoch01-accuracy0.88588.tf
INFO:tensorflow:Assets written to: model5.epoch01-accuracy0.88588.tf\assets


INFO:tensorflow:Assets written to: model5.epoch01-accuracy0.88588.tf\assets


758/758 [==============================] - 1913s 3s/step - loss: 0.4076 - accuracy: 0.8160 - f1_m: 0.5002 - precision_m: 0.3900 - recall_m: 0.7694 - val_loss: 0.2792 - val_accuracy: 0.8859 - val_f1_m: 0.5648 - val_precision_m: 0.4123 - val_recall_m: 0.9419
Epoch 2/3
758/758 [==============================] - ETA: 0s - loss: 0.2272 - accuracy: 0.9143 - f1_m: 0.5487 - precision_m: 0.4020 - recall_m: 0.9253
Epoch 2: val_accuracy improved from 0.88588 to 0.89248, saving model to model5.epoch02-accuracy0.89248.tf
INFO:tensorflow:Assets written to: model5.epoch02-accuracy0.89248.tf\assets


INFO:tensorflow:Assets written to: model5.epoch02-accuracy0.89248.tf\assets


758/758 [==============================] - 2093s 3s/step - loss: 0.2272 - accuracy: 0.9143 - f1_m: 0.5487 - precision_m: 0.4020 - recall_m: 0.9253 - val_loss: 0.2778 - val_accuracy: 0.8925 - val_f1_m: 0.5752 - val_precision_m: 0.4181 - val_recall_m: 0.9630
Epoch 3/3
758/758 [==============================] - ETA: 0s - loss: 0.1646 - accuracy: 0.9419 - f1_m: 0.5524 - precision_m: 0.4019 - recall_m: 0.9429
Epoch 3: val_accuracy improved from 0.89248 to 0.90699, saving model to model5.epoch03-accuracy0.90699.tf
INFO:tensorflow:Assets written to: model5.epoch03-accuracy0.90699.tf\assets


INFO:tensorflow:Assets written to: model5.epoch03-accuracy0.90699.tf\assets


758/758 [==============================] - 2434s 3s/step - loss: 0.1646 - accuracy: 0.9419 - f1_m: 0.5524 - precision_m: 0.4019 - recall_m: 0.9429 - val_loss: 0.2503 - val_accuracy: 0.9070 - val_f1_m: 0.5680 - val_precision_m: 0.4108 - val_recall_m: 0.9671


In [70]:
val_acc = model5.history.history['val_accuracy']
val_f1_m = model5.history.history['val_f1_m']
print(val_acc)
print(val_f1_m)

[0.8858839273452759, 0.892480194568634, 0.9069920778274536]
[0.5647614598274231, 0.5751519203186035, 0.567965030670166]


# MODEL 6

In [26]:
batch_size = 16
num_epochs = 5
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [27]:
with tf.device('/CPU:0'):
    model6 = TFAutoModelForSequenceClassification.from_pretrained(
        "bert-large-uncased", num_labels=2, id2label=id2label, label2id=label2id
    )

    model6.summary()

    tf_train_set6 = model6.prepare_tf_dataset(
        tokenized_train_ds,
        shuffle=True,
        batch_size=16,
        collate_fn=data_collator,
    )

    tf_validation_set6 = model6.prepare_tf_dataset(
        tekenized_validation_ds,
        shuffle=False,
        batch_size=16,
        collate_fn=data_collator,
    )
    model6.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m]) # No loss argument!
    # metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set6)
    push_to_hub_callback = PushToHubCallback(
        output_dir="IC_sexto",
        tokenizer=tokenizer,
    )
    checkpoint = ModelCheckpoint(filepath = 'model6.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                                monitor = 'val_accuracy',
                                verbose = 1,
                                save_best_only = True,
                                mode = 'max')
    callbacks = [checkpoint, push_to_hub_callback]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  335141888 
                                                                 
 dropout_73 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  2050      
                                                                 
Total params: 335,143,938
Trainable params: 335,143,938
Non-trainable params: 0
_________________________________________________________________


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
c:\Users\gusta\Documents\IC\IC_sexto is already a clone of https://huggingface.co/gustavokpc/IC_sexto. Make sure you pull the latest changes with `repo.git_pull()`.


In [28]:
with tf.device('/CPU:0'):
    model6.fit(x=tf_train_set6, validation_data=tf_validation_set6, epochs=num_epochs, callbacks=callbacks)

Epoch 1/5
758/758 [==============================] - ETA: 0s - loss: 0.3898 - accuracy: 0.8294 - f1_m: 0.3411 - precision_m: 0.2894 - recall_m: 0.4810
Epoch 1: val_accuracy improved from -inf to 0.89842, saving model to model6.epoch01-accuracy0.89842.tf


INFO:tensorflow:Assets written to: model6.epoch01-accuracy0.89842.tf\assets


INFO:tensorflow:Assets written to: model6.epoch01-accuracy0.89842.tf\assets


758/758 [==============================] - 2857s 4s/step - loss: 0.3898 - accuracy: 0.8294 - f1_m: 0.3411 - precision_m: 0.2894 - recall_m: 0.4810 - val_loss: 0.2440 - val_accuracy: 0.8984 - val_f1_m: 0.5087 - val_precision_m: 0.3814 - val_recall_m: 0.8079
Epoch 2/5
758/758 [==============================] - ETA: 0s - loss: 0.2070 - accuracy: 0.9228 - f1_m: 0.4927 - precision_m: 0.3723 - recall_m: 0.7869
Epoch 2: val_accuracy improved from 0.89842 to 0.92678, saving model to model6.epoch02-accuracy0.92678.tf


INFO:tensorflow:Assets written to: model6.epoch02-accuracy0.92678.tf\assets


INFO:tensorflow:Assets written to: model6.epoch02-accuracy0.92678.tf\assets


758/758 [==============================] - 2899s 4s/step - loss: 0.2070 - accuracy: 0.9228 - f1_m: 0.4927 - precision_m: 0.3723 - recall_m: 0.7869 - val_loss: 0.1911 - val_accuracy: 0.9268 - val_f1_m: 0.5222 - val_precision_m: 0.3853 - val_recall_m: 0.8520
Epoch 3/5
758/758 [==============================] - ETA: 0s - loss: 0.1392 - accuracy: 0.9467 - f1_m: 0.5266 - precision_m: 0.3881 - recall_m: 0.8670
Epoch 3: val_accuracy did not improve from 0.92678
758/758 [==============================] - 4581s 6s/step - loss: 0.1392 - accuracy: 0.9467 - f1_m: 0.5266 - precision_m: 0.3881 - recall_m: 0.8670 - val_loss: 0.2310 - val_accuracy: 0.9057 - val_f1_m: 0.5617 - val_precision_m: 0.4162 - val_recall_m: 0.9092
Epoch 4/5
758/758 [==============================] - ETA: 0s - loss: 0.1136 - accuracy: 0.9570 - f1_m: 0.5387 - precision_m: 0.3946 - recall_m: 0.9100
Epoch 4: val_accuracy did not improve from 0.92678
758/758 [==============================] - 5044s 7s/step - loss: 0.1136 - accuracy

In [ ]:
val_acc = model6.history.history['val_accuracy']
val_f1_m = model6.history.history['val_f1_m']
print(val_acc)
print(val_f1_m)

[0.9096305966377258, 0.9274406433105469, 0.9228231906890869, 0.9221636056900024, 0.9215039610862732]
[0.5641880035400391, 0.5641904473304749, 0.5681005120277405, 0.5714690089225769, 0.5709841847419739]


# MODEL 7

In [25]:
batch_size = 16
num_epochs = 3
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [26]:
with tf.device('/CPU:0'):
    model7 = TFAutoModelForSequenceClassification.from_pretrained(
        "bert-large-uncased", num_labels=2, id2label=id2label, label2id=label2id
    )

    tf_train_set7 = model7.prepare_tf_dataset(
        tokenized_train_ds,
        shuffle=True,
        batch_size=16,
        collate_fn=data_collator,
    )

    tf_validation_set7 = model7.prepare_tf_dataset(
        tekenized_validation_ds,
        shuffle=False,
        batch_size=16,
        collate_fn=data_collator,
    )
    model7.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m])  # No loss argument!
    # metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set7)
    push_to_hub_callback = PushToHubCallback(
        output_dir="IC_setimo",
        tokenizer=tokenizer,
    )
    checkpoint = ModelCheckpoint(filepath = 'model7.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                                monitor = 'val_accuracy',
                                verbose = 1,
                                save_best_only = True,
                                mode = 'max')
    callbacks = [checkpoint, push_to_hub_callback]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
c:\Users\gusta\Documents\IC\IC_setimo is already a clone of https://huggingface.co/gustavokpc/IC_setimo. Make sure you pull the latest changes with `repo.git_pull()`.


In [27]:
with tf.device('/CPU:0'):
    model7.fit(x=tf_train_set7, validation_data=tf_validation_set7, epochs=num_epochs, callbacks=callbacks)

Epoch 1/3
758/758 [==============================] - ETA: 0s - loss: 0.4176 - accuracy: 0.8080 - f1_m: 0.4543 - precision_m: 0.3667 - recall_m: 0.6857
Epoch 1: val_accuracy improved from -inf to 0.89908, saving model to model7.epoch01-accuracy0.89908.tf


INFO:tensorflow:Assets written to: model7.epoch01-accuracy0.89908.tf\assets


INFO:tensorflow:Assets written to: model7.epoch01-accuracy0.89908.tf\assets
Several commits (2) will be pushed upstream.


758/758 [==============================] - 2860s 4s/step - loss: 0.4176 - accuracy: 0.8080 - f1_m: 0.4543 - precision_m: 0.3667 - recall_m: 0.6857 - val_loss: 0.2600 - val_accuracy: 0.8991 - val_f1_m: 0.5567 - val_precision_m: 0.4108 - val_recall_m: 0.9084
Epoch 2/3
758/758 [==============================] - ETA: 0s - loss: 0.2122 - accuracy: 0.9203 - f1_m: 0.5400 - precision_m: 0.3991 - recall_m: 0.8908
Epoch 2: val_accuracy improved from 0.89908 to 0.92150, saving model to model7.epoch02-accuracy0.92150.tf


INFO:tensorflow:Assets written to: model7.epoch02-accuracy0.92150.tf\assets


INFO:tensorflow:Assets written to: model7.epoch02-accuracy0.92150.tf\assets


758/758 [==============================] - 2967s 4s/step - loss: 0.2122 - accuracy: 0.9203 - f1_m: 0.5400 - precision_m: 0.3991 - recall_m: 0.8908 - val_loss: 0.2049 - val_accuracy: 0.9215 - val_f1_m: 0.5529 - val_precision_m: 0.4068 - val_recall_m: 0.9089
Epoch 3/3
758/758 [==============================] - ETA: 0s - loss: 0.1339 - accuracy: 0.9523 - f1_m: 0.5559 - precision_m: 0.4041 - recall_m: 0.9513
Epoch 3: val_accuracy improved from 0.92150 to 0.92216, saving model to model7.epoch03-accuracy0.92216.tf


INFO:tensorflow:Assets written to: model7.epoch03-accuracy0.92216.tf\assets


INFO:tensorflow:Assets written to: model7.epoch03-accuracy0.92216.tf\assets


758/758 [==============================] - 4082s 5s/step - loss: 0.1339 - accuracy: 0.9523 - f1_m: 0.5559 - precision_m: 0.4041 - recall_m: 0.9513 - val_loss: 0.2110 - val_accuracy: 0.9222 - val_f1_m: 0.5681 - val_precision_m: 0.4137 - val_recall_m: 0.9574


In [28]:
val_acc = model7.history.history['val_accuracy']
val_f1_m = model7.history.history['val_f1_m']
print(val_acc)
print(val_f1_m)

[0.899076521396637, 0.9215039610862732, 0.9221636056900024]
[0.5567145347595215, 0.5529072284698486, 0.568143367767334]


# MODEL 8

In [32]:
batch_size = 16
num_epochs = 3
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [33]:
model8 = TFAutoModelForSequenceClassification.from_pretrained(
    "roberta-base", num_labels=2, id2label=id2label, label2id=label2id
)

tf_train_set8 = model8.prepare_tf_dataset(
    tokenized_train_ds,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set8 = model8.prepare_tf_dataset(
    tekenized_validation_ds,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
model8.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m])  # No loss argument!
# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set8)
push_to_hub_callback = PushToHubCallback(
    output_dir="IC_oitavo",
    tokenizer=tokenizer,
)
checkpoint = ModelCheckpoint(filepath = 'model8.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                             monitor = 'val_accuracy',
                             verbose = 1,
                             save_best_only = True,
                             mode = 'max')
callbacks = [checkpoint, push_to_hub_callback]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [34]:
model8.fit(x=tf_train_set8, validation_data=tf_validation_set8,--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- epochs=num_epochs, callbacks=callbacks)

Epoch 1/3


ResourceExhaustedError: Graph execution error:

Detected at node 'Adam/Adam/update_192/mul_1' defined at (most recent call last):
    File "c:\Users\gusta\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\gusta\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\gusta\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\gusta\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1053, in launch_instance
      app.start()
    File "C:\Users\gusta\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 736, in start
      self.io_loop.start()
    File "C:\Users\gusta\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py", line 195, in start
      sel

In [29]:
val_acc = model8.history.history['val_accuracy']
val_f1_m = model8.history.history['val_f1_m']
print(val_acc)
print(val_f1_m)

[0.8891820311546326, 0.9142480492591858, 0.9135884046554565]
[0.5707703232765198, 0.5582769513130188, 0.5594871640205383]


# MODEL 9

In [30]:
batch_size = 16
num_epochs = 5
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [31]:
model9 = TFAutoModelForSequenceClassification.from_pretrained(
    "roberta-base", num_labels=2, id2label=id2label, label2id=label2id
)

tf_train_set9 = model9.prepare_tf_dataset(
    tokenized_train_ds,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set9 = model9.prepare_tf_dataset(
    tekenized_validation_ds,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
model9.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m])  # No loss argument!
# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set9)
push_to_hub_callback = PushToHubCallback(
    output_dir="IC_nono",
    tokenizer=tokenizer,
)
checkpoint = ModelCheckpoint(filepath = 'model9.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                             monitor = 'val_accuracy',
                             verbose = 1,
                             save_best_only = True,
                             mode = 'max')
callbacks = [checkpoint, push_to_hub_callback]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [32]:
model9.fit(x=tf_train_set9, validation_data=tf_validation_set9, epochs=num_epochs, callbacks=callbacks)

Epoch 1/5
758/758 [==============================] - ETA: 0s - loss: 0.3792 - accuracy: 0.8256 - f1_m: 0.4970 - precision_m: 0.3813 - recall_m: 0.7827

Several commits (2) will be pushed upstream.


758/758 [==============================] - 1444s 2s/step - loss: 0.3792 - accuracy: 0.8256 - f1_m: 0.4970 - precision_m: 0.3813 - recall_m: 0.7827 - val_loss: 0.2438 - val_accuracy: 0.9116 - val_f1_m: 0.5469 - val_precision_m: 0.3963 - val_recall_m: 0.9240
Epoch 2/5
758/758 [==============================] - 1377s 2s/step - loss: 0.2147 - accuracy: 0.9128 - f1_m: 0.5383 - precision_m: 0.3958 - recall_m: 0.8972 - val_loss: 0.2278 - val_accuracy: 0.9109 - val_f1_m: 0.5656 - val_precision_m: 0.4104 - val_recall_m: 0.9530
Epoch 3/5
758/758 [==============================] - 1310s 2s/step - loss: 0.1597 - accuracy: 0.9357 - f1_m: 0.5504 - precision_m: 0.4004 - recall_m: 0.9353 - val_loss: 0.1952 - val_accuracy: 0.9228 - val_f1_m: 0.5613 - val_precision_m: 0.4056 - val_recall_m: 0.9564
Epoch 4/5
758/758 [==============================] - 1261s 2s/step - loss: 0.1200 - accuracy: 0.9530 - f1_m: 0.5532 - precision_m: 0.4016 - recall_m: 0.9526 - val_loss: 0.1926 - val_accuracy: 0.9294 - val_f1_m

KeyboardInterrupt: 

In [ ]:
val_acc = model9.history.history['val_accuracy']
val_f1_m = model9.history.history['val_f1_m']
print(val_acc)
print(val_f1_m)

# MODEL 10

In [29]:
batch_size = 16
num_epochs = 7
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=1e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [30]:
model10 = TFAutoModelForSequenceClassification.from_pretrained(
    "bert-large-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

tf_train_set10 = model10.prepare_tf_dataset(
    tokenized_train_ds,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set10 = model10.prepare_tf_dataset(
    tekenized_validation_ds,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
model10.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m])  # No loss argument!
# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set10)
push_to_hub_callback = PushToHubCallback(
    output_dir="IC_10",
    tokenizer=tokenizer,
)
checkpoint = ModelCheckpoint(filepath = 'model10.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                             monitor = 'val_accuracy',
                             verbose = 1,
                             save_best_only = True,
                             mode = 'max')
callbacks = [checkpoint, push_to_hub_callback]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\gusta\Documents\IC\IC_10 is already a clone of https://huggingface.co/gustavokpc/IC_10. Make sure you pull the latest changes with `repo.git_pull()`.


In [31]:
model10.fit(x=tf_train_set10, validation_data=tf_validation_set10, epochs=num_epochs, callbacks=callbacks)

Epoch 1/7


ResourceExhaustedError: Graph execution error:

Detected at node 'tf_bert_for_sequence_classification_1/bert/encoder/layer_._8/attention/self/transpose' defined at (most recent call last):
    File "c:\Users\gusta\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\gusta\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\gusta\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\gusta\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1053, in launch_instance
      app.start()
    File "C:\Users\gusta\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 736, in start
      self.io_loop.start()
    File "C:\Users\gusta\AppData\Roaming\Python\Python310\site-packages

In [ ]:
val_acc = model10.history.history['val_accuracy']
val_f1_m = model10.history.history['val_f1_m']
print(val_acc)
print(val_f1_m)

[0.8608179688453674, 0.892480194568634, 0.916886568069458, 0.9221636056900024, 0.9109498858451843, 0.9155672788619995, 0.920844316482544]
[0.5444657802581787, 0.5735011100769043, 0.5532918572425842, 0.5650185942649841, 0.5663719177246094, 0.5573129057884216, 0.5605470538139343]


# MODEL 11

In [40]:
batch_size = 16
num_epochs = 7
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=5e-6, num_warmup_steps=0, num_train_steps=total_train_steps)

In [41]:
model11 = TFAutoModelForSequenceClassification.from_pretrained(
    "bert-large-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

tf_train_set11 = model11.prepare_tf_dataset(
    tokenized_train_ds,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set11 = model11.prepare_tf_dataset(
    tekenized_validation_ds,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
model11.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m])  # No loss argument!
# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set11)
push_to_hub_callback = PushToHubCallback(
    output_dir="IC_11",
    tokenizer=tokenizer,
)
checkpoint = ModelCheckpoint(filepath = 'model11.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                             monitor = 'val_accuracy',
                             verbose = 1,
                             save_best_only = True,
                             mode = 'max')
callbacks = [checkpoint, push_to_hub_callback]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Cloning https://huggingface.co/gustavokpc/IC_11 into local empty directory.


Download file tf_model.h5:   0%|          | 16.0k/1.25G [00:00<?, ?B/s]

Clean file tf_model.h5:   0%|          | 1.00k/1.25G [00:00<?, ?B/s]

In [42]:
model11.fit(x=tf_train_set11, validation_data=tf_validation_set11, epochs=num_epochs, callbacks=callbacks)

Epoch 1/7
758/758 [==============================] - 3910s 5s/step - loss: 0.4434 - accuracy: 0.8014 - f1_m: 0.4236 - precision_m: 0.3661 - recall_m: 0.5865 - val_loss: 0.2743 - val_accuracy: 0.8872 - val_f1_m: 0.4964 - val_precision_m: 0.3787 - val_recall_m: 0.7644
Epoch 2/7
758/758 [==============================] - 3755s 5s/step - loss: 0.2804 - accuracy: 0.8901 - f1_m: 0.4898 - precision_m: 0.3778 - recall_m: 0.7488 - val_loss: 0.2824 - val_accuracy: 0.8879 - val_f1_m: 0.5567 - val_precision_m: 0.4181 - val_recall_m: 0.8782
Epoch 3/7
758/758 [==============================] - 3752s 5s/step - loss: 0.2254 - accuracy: 0.9128 - f1_m: 0.5069 - precision_m: 0.3838 - recall_m: 0.8028 - val_loss: 0.2388 - val_accuracy: 0.9090 - val_f1_m: 0.5468 - val_precision_m: 0.4009 - val_recall_m: 0.9053
Epoch 4/7
758/758 [==============================] - 3725s 5s/step - loss: 0.1873 - accuracy: 0.9303 - f1_m: 0.5203 - precision_m: 0.3889 - recall_m: 0.8490 - val_loss: 0.2200 - val_accuracy: 0.9149 

In [43]:
val_acc = model11.history.history['val_accuracy']
val_f1_m = model11.history.history['val_f1_m']
print(val_acc)
print(val_f1_m)

[0.8872031569480896, 0.8878628015518188, 0.9089709520339966, 0.9149076342582703, 0.9234828352928162, 0.9228231906890869, 0.9201846718788147]
[0.49644768238067627, 0.5567076206207275, 0.5467871427536011, 0.5560939908027649, 0.5565869212150574, 0.5505979061126709, 0.5513056516647339]


# MODEL 12

In [42]:
batch_size = 16
num_epochs = 5
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=5e-6, num_warmup_steps=0, num_train_steps=total_train_steps)

In [43]:
model12 = TFAutoModelForSequenceClassification.from_pretrained(
    "roberta-large", num_labels=2, id2label=id2label, label2id=label2id
)

tf_train_set12 = model12.prepare_tf_dataset(
    tokenized_train_ds,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set12 = model12.prepare_tf_dataset(
    tekenized_validation_ds,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
model12.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m])  # No loss argument!
# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set12)
push_to_hub_callback = PushToHubCallback(
    output_dir="IC_12",
    tokenizer=tokenizer,
)
checkpoint = ModelCheckpoint(filepath = 'model12.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                             monitor = 'val_accuracy',
                             verbose = 1,
                             save_best_only = True,
                             mode = 'max')
callbacks = [checkpoint, push_to_hub_callback]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [44]:
model12.fit(x=tf_train_set12, validation_data=tf_validation_set12, epochs=num_epochs, callbacks=callbacks)

Epoch 1/5
663/758 [=========================>....] - ETA: 7:56 - loss: 0.4071 - accuracy: 0.8133 - f1_m: 0.4281 - precision_m: 0.3614 - recall_m: 0.6192

KeyboardInterrupt: 

In [ ]:
val_acc = model12.history.history['val_accuracy']
val_f1_m = model12.history.history['val_f1_m']
print(val_acc)
print(val_f1_m)

# MODEL 13

In [ ]:
batch_size = 16
num_epochs = 3
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
model13 = TFAutoModelForSequenceClassification.from_pretrained(
    "roberta-large", num_labels=2, id2label=id2label, label2id=label2id
)

tf_train_set13 = model13.prepare_tf_dataset(
    tokenized_train_ds,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set13 = model13.prepare_tf_dataset(
    tekenized_validation_ds,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
model13.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m])  # No loss argument!
# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set13)
push_to_hub_callback = PushToHubCallback(
    output_dir="IC_13",
    tokenizer=tokenizer,
)
checkpoint = ModelCheckpoint(filepath = 'model13.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                             monitor = 'val_accuracy',
                             verbose = 1,
                             save_best_only = True,
                             mode = 'max')
callbacks = [checkpoint, push_to_hub_callback]

In [ ]:
model13.fit(x=tf_train_set13, validation_data=tf_validation_set13, epochs=num_epochs, callbacks=callbacks)

In [ ]:
val_acc = model13.history.history['val_accuracy']
val_f1_m = model13.history.history['val_f1_m']
print(val_acc)
print(val_f1_m)

# MODEL 14

In [ ]:
batch_size = 16
num_epochs = 3
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=6e-6, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
model14 = TFAutoModelForSequenceClassification.from_pretrained(
    "roberta-large", num_labels=2, id2label=id2label, label2id=label2id
)

tf_train_set14 = model14.prepare_tf_dataset(
    tokenized_train_ds,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set14 = model14.prepare_tf_dataset(
    tekenized_validation_ds,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
model14.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m])  # No loss argument!
# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set14)
push_to_hub_callback = PushToHubCallback(
    output_dir="IC_14",
    tokenizer=tokenizer,
)
checkpoint = ModelCheckpoint(filepath = 'model14.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                             monitor = 'val_accuracy',
                             verbose = 1,
                             save_best_only = True,
                             mode = 'max')
callbacks = [checkpoint, push_to_hub_callback]

In [ ]:
model14.fit(x=tf_train_set14, validation_data=tf_validation_set14, epochs=num_epochs, callbacks=callbacks)

In [ ]:
val_acc = model14.history.history['val_accuracy']
val_f1_m = model14.history.history['val_f1_m']
print(val_acc)
print(val_f1_m)